In [1]:
import pandas as pd
import os, sys
sys.path.insert(0, "\\".join(os.path.abspath(os.curdir).split("\\")[:-2:]))
from pathlib import Path

from src.utils.utils import *

# Pré processar os metadados

In [2]:
from copy import deepcopy

def rename_keys(dictionary):
    dict_keys_to_rename = {
        "id1": "patient_id",
        "leftright": "left_or_right_breast",
        "abnormality": "abnormality_type",
        "classification": "pathology",
        "reference_number": "patient_id",
        "laterality": "left_or_right breast",
        "view": "image_view",
        "assessment": "bi-rads",
        "age": "patient_age",
        "acr": "breast_density"
    }

    keys_order = dictionary.keys()
    
    for key in dictionary.keys():
        key_lower = deepcopy(key)
        key_lower = key_lower.lower().replace(" ", "_")
        
        if key in dict_keys_to_rename.keys():
            keys_order = list(map(lambda x: x.replace(key, dict_keys_to_rename[key_lower]), keys_order))
            
    new_dict = {}
    for key_order, key in zip(keys_order, dictionary.keys()):
        new_dict[key_order.lower()] = dictionary[key]
           
    return new_dict

In [3]:
studies = []

#DDSM
json_names = ['calc_case_description_test_set', 'calc_case_description_train_set', 
              'mass_case_description_test_set', 'mass_case_description_train_set']

for json_name in json_names:
    list_metadata = load_json(json_name, "../../metadata/extracted_metadata/csv_and_dicom_DSSM")
    
    for metadata in list_metadata:
        new_metadata = {}
        new_metadata['database_name'] = 'DDSM'
        new_metadata['study_name'] = metadata['study_name']
        
        metadata_csv = rename_keys(metadata['metadata_csv'])
        
        new_metadata = new_metadata | metadata_csv
        new_metadata['metadata_dicom_files'] = metadata['metadata_dicom_files']
        
        studies.append(new_metadata)
        
#CMMD e INBREAST     
database_names = ['CMMD', 'INBREAST']
json_names = ['csv_and_dicom_CMMD', 'csv_and_dicom_INBREAST']

for database_name, json_name in zip(database_names, json_names):
    list_metadata = load_json(json_name, "../../metadata/extracted_metadata")
    
    for metadata in list_metadata: 
        new_metadata = {}
        new_metadata['database_name'] = database_name
        new_metadata['study_name'] = metadata['study_name']
        
        metadata_csv = rename_keys(metadata['metadata_csv'])
    
        new_metadata = new_metadata | metadata_csv
        new_metadata['metadata_dicom_files'] = metadata['metadata_dicom_files']
        
        studies.append(new_metadata)
    
save_json("metadata", studies, "../../metadata/mongodb_database")